In [157]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

In [158]:
df = pd.read_csv('data cleaning assignment')

In [159]:
df.dropna(axis=0, subset=['Journal title'], inplace=True) 
#Dropped one row, because journal title was missing.
#There were no other Journals from the publisher, so replacing the space with the most common title was not possible.

In [160]:
df['Journal title'] = df['Journal title'].str.upper()
df['Journal title'] = df['Journal title'].str.strip()
df['Publisher'] = df['Publisher'].str.upper()
df['Publisher'] = df['Publisher'].str.strip()
df['Article title'] = df['Article title'].str.upper()
df['Article title'] = df['Article title'].str.strip()


def clean_plos(journal): 
    # Search for Journal titles with PLOS 
    if re.search('PLOS.*', journal): 
          # return the cleaned name 
        return 'PLOS'
    else: 
        # if clean up not needed return the same name 
        return journal
df['Journal title'] = df['Journal title'].apply(clean_plos)

In [161]:
#Clean all variations of "JOURNAL OF BIOLOGICAL CHEMISTRY"
df['Journal title'] = df['Journal title'].replace('THE JOURNAL OF BIOLOGICAL CHEMISTRY', 'JOURNAL OF BIOLOGICAL CHEMISTRY')
df['Journal title'] = df['Journal title'].replace('J BIOL CHEM.', 'JOURNAL OF BIOLOGICAL CHEMISTRY')
df['Journal title'] = df['Journal title'].replace('JNL BIOLOGICAL CHEMISTRY', 'JOURNAL OF BIOLOGICAL CHEMISTRY')
df['Journal title'] = df['Journal title'].replace('J BIOL CHEMISTRY', 'JOURNAL OF BIOLOGICAL CHEMISTRY')
df['Journal title'] = df['Journal title'].replace('JOURNAL BIOLOGICAL CHEMISTRY', 'JOURNAL OF BIOLOGICAL CHEMISTRY')
df['Journal title'] = df['Journal title'].replace('JOURNAL OF BIOL CHEM', 'JOURNAL OF BIOLOGICAL CHEMISTRY')

# Top 5 Journals

In [162]:
top_5_journals = df['Journal title'].value_counts().head(5)
print(top_5_journals)

PLOS                                               298
JOURNAL OF BIOLOGICAL CHEMISTRY                     71
NEUROIMAGE                                          29
NUCLEIC ACIDS RESEARCH                              26
PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES     22
Name: Journal title, dtype: int64


In [163]:
def usd_lb_conversion(value):
    if re.search('£', value):
        value = value.replace('£', '')
        value = float(value)*1.3
    elif re.search('\$', value):
        value = value.replace('$', '')
        value = float(value)
    else:
        value = float(value)
    return value

df['cost_in_USD'] = df['COST (£) charged to Wellcome (inc VAT when charged)'].apply(usd_lb_conversion)
df.head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged),cost_in_USD
0,NaN,CUP,PSYCHOLOGICAL MEDICINE,REDUCED PARAHIPPOCAMPAL CORTICAL THICKNESS IN ...,£0.00,0.000
1,PMC3679557,ACS,BIOMACROMOLECULES,STRUCTURAL CHARACTERIZATION OF A MODEL GRAM-NE...,£2381.04,3095.352
2,23043264 PMC3506128,ACS,J MED CHEM,"FUMAROYLAMINO-4,5-EPOXYMORPHINANS AND RELATED ...",£642.56,835.328
3,23438330 PMC3646402,ACS,J MED CHEM,ORVINOLS WITH MIXED KAPPA/MU OPIOID RECEPTOR A...,£669.64,870.532
4,23438216 PMC3601604,ACS,J ORG CHEM,REGIOSELECTIVE OPENING OF MYO-INOSITOL ORTHOES...,£685.88,891.644


In [164]:
df.drop(df[df['cost_in_USD'] > 11000].index, inplace=True)
df['cost_in_USD'].sort_values(ascending=False)

1619    7800.000
800     7488.000
552     6240.000
798     6240.000
648     6240.000
          ...   
1677     175.877
1469     159.003
100       59.722
243       12.909
0          0.000
Name: cost_in_USD, Length: 2077, dtype: float64

In [165]:
df.dropna(axis=0, subset=['Article title'], inplace=True)

# Top 5 journals cost summary

In [180]:
results = pd.DataFrame()
results['PLOS'] = round(df[df['Journal title'] == 'PLOS'].describe()['cost_in_USD'], 2)
results['JOURNAL OF BIO CHEM'] = round(df[df['Journal title'] == 'JOURNAL OF BIOLOGICAL CHEMISTRY'].describe()['cost_in_USD'], 2)
results['NEUROIMAGE'] = round(df[df['Journal title'] == 'NEUROIMAGE'].describe()['cost_in_USD'], 2)
results['NUCLEIC ACIDS RESEARCH'] = round(df[df['Journal title'] == 'NUCLEIC ACIDS RESEARCH'].describe()['cost_in_USD'], 2)
results['PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES'] = round(df[df['Journal title'] == 'PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES'].describe()['cost_in_USD'], 2)
results

,PLOS,JOURNAL OF BIO CHEM,NEUROIMAGE,NUCLEIC ACIDS RESEARCH,PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES
count,284.00,69.00,29.00,26.00,22.00
mean,1436.17,1801.53,2879.72,1493.70,1030.92
std,447.93,507.63,346.65,575.82,603.55
min,159.00,345.37,2271.31,923.00,268.22
25%,1131.17,1498.54,2730.70,1107.60,817.04
50%,1318.69,1708.89,3024.36,1107.60,964.66
75%,1814.91,2043.91,3115.54,2215.20,1020.70
max,2455.70,3251.39,3254.34,2839.20,3499.18
